<a href="https://colab.research.google.com/github/vothane/tripleplay/blob/master/tripleplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import numpy as np
import os.path

In [71]:
f = lambda img_file : img_to_array(load_img('imgs/{}'.format(img_file), color_mode = "grayscale"))
pitcher_map = {img_file[:-4] : f(img_file)
              for img_file in os.listdir('imgs/')}
print(np.shape(pitcher_map['Chris_Sale']))

(600, 600, 1)


In [72]:
image_size = 600 # x=y

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
latent_dim = 16
# encoder/decoder number of CNN layers and filters per layer
layer_filters = [32, 64]

# build the autoencoder model
# first build the encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs

# stack of Conv2D(32)-Conv2D(64)
for filters in layer_filters:
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=2,
               activation='relu',
               padding='same')(x)

# shape info needed to build decoder model so we don't do hand computation
# the input to the decoder's first Conv2DTranspose will have this shape
shape = K.int_shape(x)

# generate the latent vector
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

# instantiate encoder model
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 600, 600, 1)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 300, 300, 32)      320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 150, 150, 64)      18496     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1440000)           0         
_________________________________________________________________
latent_vector (Dense)        (None, 16)                23040016  
Total params: 23,058,832
Trainable params: 23,058,832
Non-trainable params: 0
_________________________________________________________________


In [73]:
# build the decoder model
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
# use the shape (7, 7, 64) that was earlier saved
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
# from vector to suitable shape for transposed conv
x = Reshape((shape[1], shape[2], shape[3]))(x)

# stack of Conv2DTranspose(64)-Conv2DTranspose(32)
for filters in layer_filters[::-1]:
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides=2,
                        activation='relu',
                        padding='same')(x)

# reconstruct the denoised input
outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          padding='same',
                          activation='sigmoid',
                          name='decoder_output')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1440000)           24480000  
_________________________________________________________________
reshape_5 (Reshape)          (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_transpose_9 (Conv2DTr (None, 300, 300, 64)      36928     
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 600, 600, 32)      18464     
_________________________________________________________________
decoder_output (Conv2DTransp (None, 600, 600, 1)       289       
Total params: 24,535,681
Trainable params: 24,535,681
Non-trainable params: 0
________________________________________________________________

In [74]:
# autoencoder = encoder + decoder
# instantiate autoencoder model
autoencoder = Model(inputs,
                    decoder(encoder(inputs)),
                    name='autoencoder')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 600, 600, 1)       0         
_________________________________________________________________
encoder (Model)              (None, 16)                23058832  
_________________________________________________________________
decoder (Model)              (None, 600, 600, 1)       24535681  
Total params: 47,594,513
Trainable params: 47,594,513
Non-trainable params: 0
_________________________________________________________________


In [77]:
# Mean Square Error (MSE) loss function, Adam optimizer
autoencoder.compile(loss='mse', optimizer='adam')

train_data = np.array([data for _, data in pitcher_map.items()])

autoencoder.fit(train_data, train_data, epochs=1, batch_size=batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
96/96 [==============================] - 103s 1s/step - loss: 1378.7513
